In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, linear_model, metrics
import pickle

In [15]:
# read the data
df = pd.read_csv('test.tsv',sep='\t')
libs = df.libs
df.head()

,libs
0,"user32.dll,kernel32.dll"
1,"cygwin1.dll,cygiconv-2.dll,cygintl-8.dll,cygz...."
2,"winmm.dll,tapi32.dll,ws2_32.dll,setupapi.dll,v..."
3,kernel32.dll
4,kernel32.dll


In [16]:
df.shape

(1200, 1)

In [28]:
# предобработка данных
# преобразуем исходные данные в матрицу
df_train = pd.read_csv('train.tsv',sep='\t')
libs_train = df_train.libs
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(','))
vectorizer.fit_transform(libs_train)

V = vectorizer.transform(libs)
X_test = V.toarray()

In [18]:
# построение прогноза для тестовой выборки

classifier = pickle.load(open('Classifier.joblib', 'rb'))
prediction_test = classifier.predict(X_test)
prediction_test[:10]

array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1], dtype=int64)

In [19]:
with open ('prediction.txt','w') as file:
    file.writelines('prediction' + '\n')
    file.writelines(str(i) + '\n' for i in prediction_test)

## Дополнительное задание

In [47]:
# пояснение к построенному прогнозу модели
with open ('threshold.txt','r') as file:
    thr = float(file.read())
predict_proba = classifier.predict_proba(X_test)[:,1]
ind_predict_proba = np.where(predict_proba > thr)

In [52]:
lines = []
for i in range(X_test.shape[0]):
    if i in ind_predict_proba[0]:
        line = 'with a {:.2%} probability, it is a virus'.format(predict_proba[i])
        lines.append(line)
    else:
        lines.append('')

In [53]:
with open ('explain.txt','w') as file:
    file.writelines(line + '\n' for line in lines)